<a href="https://colab.research.google.com/github/yoshimethod/bioinformatics/blob/main/bioinformatics_pr7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install git+https://github.com/hmdlab/raptgen.git@v1.0.0
!sudo apt-get install fonts-noto-cjk
!pip install biopython CairoSVG svgwrite GPy==1.9.9 GPyOpt==1.2.6 scipy==1.4.1

  Cloning https://github.com/hmdlab/raptgen.git (to revision v1.0.0) to /tmp/pip-req-build-hrdpwtwq
  Running command git clone --filter=blob:none --quiet https://github.com/hmdlab/raptgen.git /tmp/pip-req-build-hrdpwtwq
  Running command git checkout -q bc2f2a58043593c0000b954fba5989a270393c5f
  Resolved https://github.com/hmdlab/raptgen.git to commit bc2f2a58043593c0000b954fba5989a270393c5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for raptgen: filename=raptgen-0.1.0-py3-none-any.whl size=22953 sha256=40b579594723c6d97bd7e62d4556b9b4ed455175029ef6601c45d6ae860e27dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-z5i4acr4/wheels/b0/fe/17/1450a3b2a5dd9e6c1ee8eeaa1e9a27d9f92152a79f509a40ed
Successfully built raptgen
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  fonts-noto-cjk-extra
The following NEW packag

In [2]:
skip_heavy_computation = True

if skip_heavy_computation:
  reload_model = True
else:
  reload_model = False

In [3]:
!wget https://raw.githubusercontent.com/hmdlab/raptgen/v1.0.0/data/simulation/paired/sequences.txt

--2024-06-07 07:06:39--  https://raw.githubusercontent.com/hmdlab/raptgen/v1.0.0/data/simulation/paired/sequences.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165000 (161K) [text/plain]
Saving to: ‘sequences.txt’

sequences.txt       100%[===================>] 161.13K  --.-KB/s    in 0.005s  

2024-06-07 07:06:39 (31.9 MB/s) - ‘sequences.txt’ saved [165000/165000]



In [4]:
!head sequences.txt

1,AAAAACATACCACACTACTGTTCTTGGGGG
0,AAAAAAACACTACCTATCCCTCTAAGGGGG
0,AAAAACACTACTCGCGCCCTCTGTGGGGGG
2,AAAAATGTACTACACCCCTCGGATAGGGGG
0,AAAAACGAAGACTACTGCCCTCGTAGGGGG
0,AAAAAGCACTACTCGGTCCCTCTTAGGGGG
0,AAAAATTATACTACCATATCCCTCCGGGGG
0,AAAAAATCAGACTACCTCCCCCTCTGGGGG
0,AAAAAGATACACTACTGTCCCTCTGGGGGG
0,AAAAAGACTACAACCCTCGACTACAGGGGG


In [6]:
!pip install biopython

  Using cached biopython-1.83-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)


In [7]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pathlib import Path

In [8]:
with Path('sequences.txt').open() as f:
  records = []
  for idx, line in enumerate(f.readlines()):
    motif_idx, sequence = line.strip().split(',')
    record = SeqRecord(
        Seq(sequence),
        id = f'{idx}',
        description = f"motif:{motif_idx} #{idx}"
    )

    records.append(record)

  SeqIO.write(
      sequences = records,
      handle ='sample.fasta',
      format = 'fasta'
  )

In [9]:
!head -n 4 sample.fasta

>0 motif:1 #0
AAAAACATACCACACTACTGTTCTTGGGGG
>1 motif:0 #1
AAAAAAACACTACCTATCCCTCTAAGGGGG


In [14]:
from raptgen.data import SingleRound
filename = 'sample.fasta'
single_round =SingleRound(path = filename)

In [15]:
import torch

epochs = 200
threshold = 10
device = "cuda"

if device == "cuda" and torch.cuda.is_available() == False:
  print("cuda is not available. using cpu.")
  device = torch.device("cpu")
else:
  print(f"using {device}.")
  device = torch.device(device)

min_count = 1

trrain_loader, test_loader = single_round.get_dataloader(
    min_count = min_count, use_cuda = (device == torch.device("cuda"))
)

force_epochs = 10
save_dir = "out/"
save_dir = Path(save_dir).expanduser()
save_dir.mkdir(exist_ok = True, parents = True)

using cuda.
